In [7]:
from torchviz import make_dot
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, DataCollatorWithPadding
from vncorenlp import VnCoreNLP
from Model.CustomSoftmaxModel import CustomModelSoftmax
from preprocessing.NewsPreprocessing import Preprocess
import torch
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
# Model 
model = CustomModelSoftmax("vinai/phobert-base")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

CustomModelSoftmax(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [ ]:
rdrsegmenter = VnCoreNLP(r"./vncorenlp_file/VnCoreNLP-1.1.1.jar",
                         annotators="wseg", max_heap_size='-Xmx500m')
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

data_files = {'train': r"../data/training_data/train_datasets.csv",
              'test': r"../data/training_data/test_datasets.csv"}
dataset = load_dataset('csv', data_files=data_files)
preprocess = Preprocess(tokenizer, rdrsegmenter)
tokenized_datasets = preprocess.run(dataset)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader = DataLoader(tokenized_datasets["train"],
                              batch_size=32,
                              collate_fn=data_collator,
                              shuffle=True)

In [35]:
tokenized_datasets['train']['input_ids'][0].shape

torch.Size([53])

In [39]:
x = torch.zeros(100, dtype=torch.float, requires_grad=False)
x.shape

torch.Size([100])

In [40]:
out = model(x)
make_dot(out)  # plot graph of variable, not of a nn.Module

IndexError: too many indices for tensor of dimension 1

torch.Size([1, 3, 224, 224])